In [23]:
import numpy as np
import pandas as pd
import string

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# read table
movies = pd.read_csv('movie_plots.csv')
movies.loc[0:4]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


# The Question
The first part of developing a data-driven project is to decide what question you want to answer. The question needs to be specific, and it needs to be something you can develop a step-by-step approach for. With this notebook, I am going to use the `movies` DataFrame to answer the following question:
> Can we predict the genre of a movie based on its plot?

It will take a few steps to answer this question. The over-arching workflow will look something like this:
1. Data preprocessing
2. Group movies by genre and look for recurring words in plots
3. Write a $k$-nearest neighbor classifier
4. Test the classifier and determine its accuracy

# Data Preprocessing
Currently, the `movies` DataFrame contains details of movies (year, title, country of origin, director, cast, genre, Wiki link, and plot). In order to get meaningful results, the data need to be "cleaned;" that is, we need to remove values that we can't work with. The data set, which contains 34,885 entries, has lots of rows that contain `NaN` values ("not a number"), which means that we can't work with them. The `pandas` library allows you to remove the rows that contain `NaN` values:

In [24]:
movies = movies.dropna().reset_index(drop=True)
movies.loc[0:4]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab..."
1,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...
2,1907,How Brown Saw the Baseball Game,American,Unknown,Unknown,comedy,https://en.wikipedia.org/wiki/How_Brown_Saw_th...,Before heading out to a baseball game at a nea...
3,1907,Laughing Gas,American,Edwin Stanton Porter,"Bertha Regustus, Edward Boulden",comedy,https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,The plot is that of a black woman going to the...
4,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...


In the above cell, we also reset the row indices to make slicing the table easier. We now have 33,464 rows, so we lost about 1,400 entries. The next thing to look at is what kinds of values we have in the DataFrame. Since we're focusing on the genre and plot, we can remove columns with irrelevant information:

In [25]:
movies = movies[['Genre', 'Plot']]
movies.loc[0:4]

,Genre,Plot
0,unknown,"Alice follows a large white rabbit down a ""Rab..."
1,biographical,Boone's daughter befriends an Indian maiden as...
2,comedy,Before heading out to a baseball game at a nea...
3,comedy,The plot is that of a black woman going to the...
4,drama,On a beautiful summer day a father and mother ...


### Data Exploration
Now we need to see what kinds of data we have in the set. We begin by showing what the values in the `Genre` column of the DataFrame are:

In [26]:
movies.groupby('Genre').agg(len)

,Plot
Genre,
,18
usa,1
"usa, can",1
-,3
16 mm film,1
[140],1
[144],1
action,1087
action,21


It looks like there's lots of different values in the column (2,210 to be exact). But for this question, we're really only interested in the basic genres of movies: action, adventure, comedy, drama, fantasy, historical, horror, romance, science fiction, and thriller.

To this end, we will define a function that will read through the `Genre` entry in each row and categorize them as one of the above. This process has a few steps:
1. Define a function that will determine if any of the above words are present in the entry
2. Filter `movies` for such rows
3. Change the entry for `Genre` to one of the above for each row left if entry is not one of the above

In [27]:
# Step 1:
def acceptable_genre(entry):
    acceptable_genres = ['action', 'adventure', 'comedy', 'drama', 'fantasy', 'historical', 'horror', 
                        'romance', 'science fiction', 'thriller']
    for genre in acceptable_genres:
        if genre in entry['Genre']:
            return True
    return False

# Step 2:
filtered_movies = movies[movies.apply(acceptable_genre, axis=1)].reset_index(drop=True)
filtered_movies.loc[0:4]

,Genre,Plot
0,comedy,Before heading out to a baseball game at a nea...
1,comedy,The plot is that of a black woman going to the...
2,drama,On a beautiful summer day a father and mother ...
3,drama,A thug accosts a girl as she leaves her workpl...
4,comedy,A young couple decides to elope after being ca...


In [28]:
# Step 3:
def superfluous_text(entry):
    """
    Determines if the string entry is in acceptable_genres
    """
    acceptable_genres = ['action', 'adventure', 'comedy', 'drama', 'fantasy', 'historical', 'horror', 
                        'romance', 'science fiction', 'thriller']
    for genre in acceptable_genres:
        if genre == entry:
            return False
    return True

def determine_genre(entry):
    """
    Takes a string entry and returns the word closest to the end that is in acceptable_genres
    """
    acceptable_genres = ['action', 'adventure', 'comedy', 'drama', 'fantasy', 'historical', 'horror', 
                        'romance', 'science fiction', 'thriller']

    if not superfluous_text(entry):
        return entry
    for c in entry:
        if c not in string.ascii_lowercase:
            genre = entry.replace(c, '')
    try:
        genre = genre.split(' ')
    except UnboundLocalError:
        genre = entry.split(' ')

    i = -1
    new_genre = ''
    while new_genre not in acceptable_genres:
        try:
            genre = genre[i]
        except IndexError:
            new_genre = ''
            return new_genre
        if genre not in acceptable_genres:
            i -= 1

    return new_genre

def change_genres(df):
    new_genres = np.array([])
    for entry in df['Genre']:
        new_genre = determine_genre(entry)
        new_genres = np.append(new_genres, new_genre)
    
    new_df = df.drop(columns=['Genre']).assign(Genre=new_genres)
    
    return new_df

filtered_genres = change_genres(filtered_movies)
filtered_genres.loc[0:4]

,Plot,Genre
0,Before heading out to a baseball game at a nea...,comedy
1,The plot is that of a black woman going to the...,comedy
2,On a beautiful summer day a father and mother ...,drama
3,A thug accosts a girl as she leaves her workpl...,drama
4,A young couple decides to elope after being ca...,comedy


### Cleaning Plot Strings


In [47]:
def clean_string(s):
    for c in s:
        if c not in string.ascii_letters + ' ':
            s = s.replace(c, '')
        elif c in string.ascii_uppercase:
            i = string.ascii_uppercase.index(c)
            s = s.replace(c, string.ascii_lowercase[i])
    return s

def clean_plots(df):
    plots = list(df['Plot'])
    
    cleaned_plots = []
    for plot in plots:
        cleaned_plot = clean_string(plot)
        cleaned_plots += [cleaned_plot]
        
    new_df = df.drop(['Plot'], axis=1).assign(Plot=cleaned_plots)
    return new_df

movies_with_cleaned_plots = clean_plots(filtered_genres)
movies_with_cleaned_plots.loc[0:4]

,Genre,Plot
0,comedy,before heading out to a baseball game at a nea...
1,comedy,the plot is that of a black woman going to the...
2,drama,on a beautiful summer day a father and mother ...
3,drama,a thug accosts a girl as she leaves her workpl...
4,comedy,a young couple decides to elope after being ca...
